In [ ]:
import sys, os, multiprocessing, urllib3, csv
from PIL import Image
from io import BytesIO
from tqdm  import tqdm
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !mkdir '/content/drive/MyDrive/AI604_TeamProject/data'
# !unzip '/content/drive/MyDrive/AI604_TeamProject/train/train.json.zip' -d /content/drive/MyDrive/data/train

In [ ]:
import pandas as pd

# use train.json zip file
train = pd.read_json('/content/drive/MyDrive/AI604_TeamProject/train/train.json')
def format_dataset(df):
    df['image_id'] = df.annotations.map(lambda x: x['image_id'])
    df['label_id'] = df.annotations.map(lambda x: x['label_id'])
    df['url'] = df.images.map(lambda x: x['url'][0])
    df.drop(columns=['annotations', 'images'], inplace=True)

format_dataset(train)

In [ ]:
import json
def ParseData(data_file):
  ann = {}
  if 'train' in data_file or 'validation' in data_file:
      _ann = json.load(open(data_file))['annotations']
      for a in _ann:
        ann[a['image_id']] = a['label_id']

  key_url_list = []
  j = json.load(open(data_file))
  images = j['images']
  for item in images:
    assert len(item['url']) == 1
    url = item['url'][0]
    id_ = item['image_id']
    # print('checking id and ann[id]', id_, ann[id_])
    # https://www.kaggle.com/andrewrib/visualizing-imaterialist-data (데이터 확인)
    # select_item = [3, 4, 6, 15, 22, 26, 45, 63, 86, 104, 119, 120 ]
    # select_item = [18, 45, 86] # 일단은 sofa위주로 학습
    select_item = [3, 15, 45, 71] # 사무실용의자, 나무의자, 소파, 세면대


    if id_ in ann:
      # print(id_)
      #104, 67, 5가 존재
      # 101=침대, 77= 에어컨/공기청정기, 74 = 그릇, 9 = 칼, 5 = 냄비, 67 = 티비
      # 18=side소파, 45 = 소파 4 = 의자, 120=쿠션의자, 104 = 높은 의자, 86 = sofabed, 15 = 나무의자, 26 = plastic chair, 3 = 사무실 의자, 22 = 소파 의자, 63 = plastic chair, 22, 119= 소파 의자
      if ann[id_] in select_item:
        id_ = "{}_{}".format(id_, ann[id_])
        key_url_list.append((id_, url))
      else:
        pass
  return key_url_list


def DownloadImage(key_url):
  out_dir = '/content/drive/MyDrive/AI604_TeamProject/final_train_data'
  (key, url) = key_url
  filename = os.path.join(out_dir, '%s.jpg' % key)


  try:
    http = urllib3.PoolManager()
    response = http.request('GET', url)
    image_data = response.data
  except:
    print('Warning: Could not download image %s from %s' % (key, url))
    return

  try:
    pil_image = Image.open(BytesIO(image_data))
  except:
    print('Warning: Failed to parse image %s %s' % (key,url))
    return

  try:
    pil_image_rgb = pil_image.convert('RGB')
  except:
    print('Warning: Failed to convert image %s to RGB' % key)
    return

  try:
    pil_image_rgb.save(filename, format='JPEG', quality=90)
  except:
    print('Warning: Failed to save image %s' % filename)
    return


key_url_list = ParseData('/content/drive/MyDrive/AI604_TeamProject/train/train.json')
pool = multiprocessing.Pool(processes=12)

with tqdm(total=len(key_url_list)) as t:
  for _ in pool.imap_unordered(DownloadImage, key_url_list):
    t.update(1)

  0%|          | 0/6981 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


  0%|          | 1/6981 [00:00<34:21,  3.39it/s]/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: Insecu

  0%|          | 3/6981 [00:01<1:01:03,  1.90it/s]/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
  0%|          | 6/6981 [00:02<41:26,  2.81it/s]/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
  0%|          | 10/6981 [

  0%|          | 11/6981 [00:02<25:09,  4.62it/s]/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
  0%|          | 12/6981 [00:02<28:47,  4.04it/s]/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
  0%|          | 15/6981 [

  0%|          | 26/6981 [00:04<18:23,  6.30it/s]/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


...

100%|█████████▉| 6978/6981 [42:28<00:47, 15.68s/it]

100%|█████████▉| 6979/6981 [42:48<00:33, 16.97s/it]

100%|█████████▉| 6980/6981 [2:21:14<00:01,  1.21s/it] Process ForkPoolWorker-10:
Process ForkPoolWorker-12:
Process ForkPoolWorker-7:
Process ForkPoolWorker-2:
Process ForkPoolWorker-11:
Process ForkPoolWorker-5:
Traceback (most recent call last):
Traceback (most recent call last):


Process ForkPoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-4:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Process ForkPoolWorker-6:
Process ForkPoolWorker-9:
Traceback (most recent call last):
Process ForkPoolWorker-1:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/mult

KeyboardInterrupt: ignored